This is where we are creating the data frames for stack spectrograms

In [1]:
import os
import numpy as np
import cv2
import librosa
import matplotlib.pyplot as plt
from skimage.transform import resize
import io


In [2]:
#Listar os vetores que possuem a terminação
folder = 'test'


In [3]:

# Definindo os parâmetros do STFT
FRAME_SIZE = 2048
HOP_SIZE = 512

# Intervalo de frequências
fmin = 15000  # 15kHz
fmax = 48000  # 48kHz

# Diretórios
base_dir = os.path.join(f'E:/backup_for_copy/{folder}/')
subdirs = ['positive', 'negative']

save_dir = 'C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/'

# Configurações da imagem
dpi = 300
width, height = 224 / dpi, 224 / dpi

In [4]:
array_final = []

In [5]:
#Teste

audio_path = 'E:/backup_for_copy/val/positive/chunk_182_PAM_MF_20190219_042000_000.wav.wav'
y, sr = librosa.load(audio_path, sr=None, mono=False)
stft = librosa.stft(y, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
y_scale = np.abs(stft) ** 2
y_log_scale = librosa.power_to_db(y_scale)

espectrogramas = []

for i in range(4):
    plt.figure(figsize=(width, height), dpi=dpi)
    plt.axis('off')
    plt.margins(0, 0)
    plt.ylim(fmin, fmax)
    
    librosa.display.specshow(y_log_scale[i, :, :],
                                sr=sr,
                                hop_length=HOP_SIZE,
                                x_axis='time',
                                y_axis='linear',
                                cmap='gray',
                                fmin=fmin,
                                fmax=fmax)
    
    plt.clim(-60, 10)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.tight_layout(pad=0)

    # Salvar a figura em um buffer de memória
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, dpi=dpi)
    buf.seek(0)
    
    # Ler a imagem do buffer e redimensionar
    img = plt.imread(buf)
    img = img[:,:,:3]#pegando só as tres dimens~~oes RGB
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#Transformando em uma dimenção grayscale
    img_resized = resize(img, (224, 224), anti_aliasing=True)
    print(img_resized.shape)
    # Adicionar o espectrograma redimensionado ao array de espectrogramas
    espectrogramas.append(img_resized[:, :])  # Use apenas o canal de luminância

    buf.close()
    plt.close()

    if i == 3 :
        espectrogramas = np.stack(espectrogramas, axis=0)

print(f'final {espectrogramas.shape}')

final = np.stack(espectrogramas,axis=0)
print(final.shape)



(224, 224)
(224, 224)
(224, 224)
(224, 224)
final (4, 224, 224)
(4, 224, 224)


In [6]:


# Função para gerar espectrogramas e salvar em um array
def generate_spectrogram(audio_path):
    y, sr = librosa.load(audio_path, sr=None, mono=False)
    stft = librosa.stft(y, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)
    y_scale = np.abs(stft) ** 2
    y_log_scale = librosa.power_to_db(y_scale)
    
    espectrogramas = []
    
    for i in range(4):
        plt.figure(figsize=(width, height), dpi=dpi)
        plt.axis('off')
        plt.margins(0, 0)
        plt.ylim(fmin, fmax)
        
        librosa.display.specshow(y_log_scale[i, :, :],
                                 sr=sr,
                                 hop_length=HOP_SIZE,
                                 x_axis='time',
                                 y_axis='linear',
                                 cmap='gray',
                                 fmin=fmin,
                                 fmax=fmax)
        
        plt.clim(-60, 10)
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        plt.tight_layout(pad=0)

        # Salvar a figura em um buffer de memória
        buf = io.BytesIO()
        plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0, dpi=dpi)
        buf.seek(0)
        
        # Ler a imagem do buffer e redimensionar
        img = plt.imread(buf)
        img = img[:,:,:3]#pegando só as tres dimenções RGB
        img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)#Transformando em uma dimenção grayscale
        img_resized = resize(img, (224, 224), anti_aliasing=True)
        #print(f'tamanho da imagem é {img_resized.shape}')
        # Adicionar o espectrograma redimensionado ao array de espectrogramas
        espectrogramas.append(img_resized[:, :])  # Use apenas o canal de luminância

        buf.close()
        plt.close()
    
    espectrogramas = np.stack(espectrogramas, axis=0)
    print(f'Isso gerou arquivos de  {espectrogramas.shape}')
    return espectrogramas

# Processar os áudios em lotes e salvar os espectrogramas
batch_size = 10  # Tamanho do lote

for subdir in subdirs:
    path = os.path.join(base_dir, subdir)
    files = [f for f in os.listdir(path) if f.endswith('.wav')]
    
    for i in range(0, len(files), batch_size):
        batch_files = files[i:i+batch_size]
        spectrogram_list = []
        
        for file in batch_files:
            
            audio_path = os.path.join(path, file)
            espectrograma = generate_spectrogram(audio_path)
            spectrogram_list.append(espectrograma)
        
        # Criar o nome do arquivo baseado no número do lote
        batch_index = i // batch_size + 1
        spectrogram_array = np.stack(spectrogram_list, axis=0)
        print(f' tamanho do shape fora do loop é {spectrogram_array.shape}')
        np.save(os.path.join(save_dir, f'gray_spectr_{folder}_{subdir}_batch_{batch_index}.npy'), spectrogram_array)

print('Processamento concluído!')


Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
 tamanho do shape fora do loop é (10, 4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
 tamanho do shape fora do loop é (10, 4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  (4, 224, 224)
Isso gerou arquivos de  

# Agora vamos juntar todos os vetores e salvar como uma coisa só


In [25]:
for i in range (len(subdirs)):
    file = f'gray_spectr_{folder}_{subdirs[i]}_'

    # Listar todos os arquivos que começam com 'gray_spectr_test_positive_'
    files = [f for f in os.listdir(save_dir) if f.startswith(file)]

    # Exibir os arquivos encontrados
    arrays = []
    for file in files:
        array = np.load(save_dir + file)
        arrays.append(array)

    concatenated_array = np.concatenate(arrays,axis=0)

    #verificando se o shape esta correto
    print(concatenated_array.shape)

    # Opcional: Salvar o array concatenado em um novo arquivo .npy

    np.save(os.path.join('E:/nps/nps_gray_stack_spec', f'{folder}_{subdirs[i]}.npy'), concatenated_array)

    print(f'{folder}_{subdirs[i]}.npy SAVED!!!')
    print(f'Seu tamanho é de {concatenated_array.shape}')

(43, 4, 224, 224)
val_positive.npy SAVED!!!
Seu tamanho é de (43, 4, 224, 224)
(43, 4, 224, 224)
val_negative.npy SAVED!!!
Seu tamanho é de (43, 4, 224, 224)


In [26]:

# Listar todos os arquivos que começam com 'test'
files_to_delete = [f for f in os.listdir(save_dir) if f.startswith(f'gray_spectr_{folder}_')]

# Deletar cada arquivo
for file_name in files_to_delete:
    file_path = os.path.join(save_dir, file_name)
    
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"Arquivo {file_path} deletado.")
    else:
        print(f"Arquivo {file_path} não encontrado.")


Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_negative_batch_1.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_negative_batch_2.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_negative_batch_3.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_negative_batch_4.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_negative_batch_5.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_saved/gray_spectr_val_positive_batch_1.npy deletado.
Arquivo C:/Users/flora/OneDrive/Documentos/MESTRADO_UFSC/rotinas/python/Flora/dolphins_append/np_sav